## GNN approach (only if brain is treated as object)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

In [ ]:
coord = pd.read_csv("../data/75_regions_coord.txt", sep="\t")
# display(coord.head())
# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')
# ax.scatter(coord.x, coord.y, coord.z)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(18900, 16)
        self.conv2 = GCNConv(16, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
full_graph = [[[i, j] for i in range(50)] for j in range(50)]
full_graph = torch.tensor(np.concatenate(full_graph).T)

data_list = []
for human in ["H1", "H2", "H3", "H5"]:
    lipid_profiles = (
        sz_molecules[sz_molecules.human == human].iloc[:, 2:-1].values.reshape(1, -1)
    )
    lipid_profiles = torch.Tensor(lipid_profiles)
    print(lipid_profiles.shape)
    data = Data(
        x=lipid_profiles,
        edge_index=full_graph,
        #         pos=coord[["x", "y", "z"]],
        y=1,
    )
    data_list.append(data)

for human in ["HA", "HB", "HC", "HD"]:
    lipid_profiles = (
        h_molecules[h_molecules.human == human].iloc[:, 2:-1].values.reshape(1, -1)
    )
    lipid_profiles = torch.Tensor(lipid_profiles)
    data = Data(
        x=lipid_profiles,
        edge_index=full_graph,
        #         pos=coord[["x", "y", "z"]],
        y=0,
    )
    print(lipid_profiles.shape)
    data_list.append(data)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
data_list

In [ ]:
loader = DataLoader(data_list, batch_size=1, shuffle=True)

In [ ]:
model(data_list[0])

In [ ]:
next(iter(loader))

In [ ]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    data = next(iter(loader)).to(device)
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()